In [ ]:
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています
# これは、kaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、以下はロードする便利なパッケージの例です

import numpy as np # 線形代数
import pandas as pd # データ処理、CSVファイルのI/O（例：pd.read_csv）

# 入力データファイルは、読み取り専用の "../input/" ディレクトリにあります
# 例えば、これを実行（「Run」をクリック、またはShift+Enter）すると、inputディレクトリ配下のすべてのファイルが一覧表示されます

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 「Save & Run All」でバージョンを作成する際、現在のディレクトリ（/kaggle/working/）に最大20GBまでファイルを書き込むことができ、それらは出力として保持されます
# /kaggle/temp/ 
